We're going to do some statistical analysis of catch data from the US East Coast. You can grab the data from
https://www.seaaroundus.org/data/#/eez/851
or from the github repository.

In [ ]:
df = read.csv("SAU EEZ 851 v50-1.csv")

In [ ]:
tail(df)

In [ ]:
library(dplyr)
df2 = df %>% group_by(year, gear_type) %>% summarize(tonnes=sum(tonnes), landed_value=sum(landed_value, na.rm=T))
write.csv(df2, "SAU EEZ 851 v50-1-gearyear.csv", row.names=F)

In [ ]:
df = read.csv("SAU EEZ 851 v50-1-gearyear.csv")

In [ ]:
head(df)

In [ ]:
unique(df$gear_type)

In [ ]:
library(ggplot2)

In [ ]:
ggplot(df, aes(x=year, y=tonnes)) +
  geom_col()

In [ ]:
ggplot(df, aes(x=year, y=tonnes, fill=gear_type)) +
  geom_col()

Question 1: What is the tonnage caught with a particular gear in a given year?

In [ ]:
df[df$year == 2000,]$tonnes

In [ ]:
df$tonnes[df$year == 2000]

In [ ]:
head(df)

In [ ]:
head(df)$gear_type == 'bagnets'

In [ ]:
sum(df$tonnes[df$year == 2000])

In [ ]:
sum(df$tonnes[df$gear_type == 'bottom trawl'])

In [ ]:
df$tonnes[df$year == 2000 & df$gear_type == 'bottom trawl']

In [ ]:
subset(df, year == 2000 & gear_type == "bottom trawl")

Question 2: What is the average value for a given gear type?

In [ ]:
subset(df, gear_type == 'bottom trawl')

In [ ]:
ggplot(subset(df, gear_type == 'bottom trawl'), aes(x=year, y=tonnes)) +
  geom_col()

In [ ]:
ggplot(subset(df, gear_type == 'bottom trawl'), aes(x=tonnes)) +
  geom_histogram()

In [ ]:
mean(df$tonnes[df$gear_type == 'bottom trawl'])

In [ ]:
quantile(df$tonnes[df$gear_type == "bottom trawl"])

Statistical model:
$$tonnes \sim \mathcal{N}(MEAN, STDDEV)$$

In [ ]:
sd(df$tonnes[df$gear_type == 'bottom trawl'])

In [ ]:
faketonnes = rnorm(1000, 415197.568343814, 318485.223031723)

In [ ]:
ggplot(data.frame(tonnes=faketonnes), aes(x=tonnes)) +
  geom_histogram()

In [ ]:
pdf1 = data.frame(type='real', tonnes=df$tonnes[df$gear_type == 'bottom trawl'])

In [ ]:
pdf2 = data.frame(type='fake', tonnes=faketonnes)

In [ ]:
pdf = rbind(pdf1, pdf2)

In [ ]:
tail(pdf)

In [ ]:
ggplot(pdf, aes(x=tonnes, colour=type)) +
  geom_density()

Statistical model:
$$log(tonnes) \sim \mathcal{N}(MEAN, STDDEV)$$

In [ ]:
mean(log(df$tonnes[df$gear_type == 'bottom trawl']))

In [ ]:
sd(log(df$tonnes[df$gear_type == 'bottom trawl']))

In [ ]:
faketonnes = exp(rnorm(1000, 12.661429700571, 0.761876733242941))

In [ ]:
pdf2 = data.frame(type='fake', tonnes=faketonnes)

In [ ]:
pdf = rbind(pdf1, pdf2)

In [ ]:
ggplot(pdf, aes(x=tonnes, colour=type)) +
  geom_density()

Question 3: Is bottom trawling declining?

In [ ]:
ggplot(subset(df, gear_type == 'bottom trawl'), aes(x=year, y=tonnes)) +
  geom_col()

Statistical model:
$$tonnes \sim \mathcal{N}(INTERCEPT + SLOPE \cdot year, STDDEV)$$

In [ ]:
lm(tonnes ~ 1 + year, subset(df, gear_type == 'bottom trawl'))

In [ ]:
summary(lm(tonnes ~ 1 + year, subset(df, gear_type == 'bottom trawl')))

In [ ]:
ggplot(subset(df, gear_type == 'bottom trawl'), aes(x=year, y=tonnes)) +
  geom_col() + geom_abline(intercept=21119880, slope=-10433)

Statistical model:
$$log(tonnes) \sim \mathcal{N}(INTERCEPT + SLOPE \cdot year, STDDEV)$$

In [ ]:
summary(lm(log(tonnes) ~ 1 + year, subset(df, gear_type == 'bottom trawl')))

In [ ]:
df$predicted = exp(74.211344 - 0.031015*df$year)

In [ ]:
ggplot(subset(df, gear_type == 'bottom trawl'), aes(x=year, y=tonnes)) +
  geom_col() + geom_line(aes(y=predicted))